In [1]:
import entity_formatter
from entity_tagger import entity_tagger as tagger
import requests
import json
import pandas as pd
import boto3
import traceback
import snorkel
import json

In [2]:
import uuid
uuid.uuid4()

UUID('175dcda3-55ba-4418-b001-44d6ca49e467')

In [3]:
import numpy as np
import spacy
import nltk
from nltk.parse import CoreNLPParser
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")
from nltk.tag.stanford import StanfordNERTagger
from nltk.tokenize import word_tokenize  
sner_tagger = StanfordNERTagger('taging_data/english.all.3class.distsim.crf.ser.gz',
               'taging_data/stanford-ner.jar',
               encoding='utf-8')

In [4]:
## MOST IMPORTANT
exp_id = "exp03" #unique for each experiment

In [5]:
ssm = boto3.client("ssm")
s3 = boto3.client("s3")
root_url = ssm.get_parameter(Name=f"/account/root-url")["Parameter"]["Value"]
apikey = ssm.get_parameter(Name="/account/internal-api-key")["Parameter"]["Value"]
v1_url = f"https://remember.{root_url}"
v2_url = f"https://rememberv2.{root_url}/latest"
acc_owner = ssm.get_parameter(Name="/account/owner")["Parameter"]["Value"].upper()
headers = {"x-api-key": apikey, "Authorization": apikey}

In [6]:
temp_tagged = []
temp_untagged = []
page_blobs = []
Full_list = []
lol = []

In [15]:
def rememberv2_query(index={}, filters={}):
    url = f"{v2_url}/query"
    results = {}
    try:
        payload = {
            "Index": index,
            "Filter": filters
        }
        results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
    except:
        print(traceback.format_exc())    
    return results


def rememberv2_read(objectid):
    url = f"{v2_url}/read"
    results = {}
    try:
        payload = {
            "ObjectId": objectid,
        }
        results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
    except:
        print(traceback.format_exc())
    return results



def remember_recall(rid, datapoint):
    url = f"{v1_url}/recall?_remember_id={rid}&_datapoint={datapoint}"
    res = {}
    try:
        res = json.loads(requests.get(url=url).text)["datapoints"][0]["data"]
    except:
        print(traceback.format_exc())
    return res
    
# def make_text_blob(word_ocr):
#     text_list = []
    
#     for i in word_ocr["Words"]:
#         text_list.append(i["text"])
#     #print("\n\n\nBefore Sending it off: " , text_list)
#     return text_list

def remember_write(datapoint):
    resp_dict = {}
    url = f"{v2_url}/write"
    try:
        resp = requests.post(
            url=url, data=json.dumps(datapoint), headers=headers
        )
        resp_dict = resp.json()
    except:
        print(traceback.format_exc())
    return resp_dict


def create_datapoint(Type, Fields, TransactionId, Attributes=None):
    datapoint = {
        "Type": Type,
        "Fields": Fields,
        "TransactionId": TransactionId,
    }
    if Attributes != None:
        datapoint["Attributes"] = Attributes
    return remember_write(datapoint)


def remember_memorize(data, rid, datapoint, metadata={}):
    url = f"{v1_url}/memoorize"
    try:
        metadata.update({
            "_remember_id": rid,
            "_datapoint": datapoint
        })
        payload = {
            "data": data,
            "metadata": metadata 
        }
        resp = requests.post(
                url=url, data=json.dumps(payload), headers=headers)
    except:
        print(traceback.format_exc())
    return resp
def do_sner_tag(text):
    text = text.replace("/","-")
    text = text.replace("[]","")
    tagged_list = sner_tagger.tag(word_tokenize(text))
    return tagged_list
def do_spacy_tag(text):
    text = text.replace("/","-")
    

def aggregate_formatted_entities(docid):
    temp_dict = {}
    try:
        recall_txn = rememberv2_read(docid)[0]
        txnid = recall_txn["TransactionId"]
        file_pages = recall_txn["Pages"]
        start = file_pages[0]
        doc_pages = list(range(1, len(file_pages)+1))
        page_ocrs_ids = {x['ParentIndex']:x['ObjectId'] for x in rememberv2_query({'PageOcr::TransactionId': txnid}, {'ParentIndex': file_pages})}
        results = {}
        formatted_doc = {}
        count = 0
        print("pages-->",sorted(page_ocrs_ids.keys()))
        for page in sorted(page_ocrs_ids.keys()):
            try:
                
                #print("Going on a count: ", count)
                count = count+1
                words_ocr = rememberv2_query({'Parent': page_ocrs_ids[page]})
                parsed_words = tagger.parse_words(words_ocr[0]['Words'])
                print(parsed_words)
                blob = tagger.make_blob(parsed_words)
                #lol.append(parsed_words)
                #print("page_number --> ", page)
                call_make_df(parsed_words,blob,page,docid)
#                 tagged = tagger.handler({'body': json.dumps(words_ocr[0])}, {})
#                 find_untagged_words(parsed_words,tagged)
#                 formatted = entity_formatter.format_entities(json.loads(tagged['body'])['entities'], page-start+1)['body']
#                 results[page] = formatted
#                 create_datapoint("PageTaggedEntitiesExp", {"Entities": formatted, "FilePageIndex": page, "ExpId": exp_id}, txnid ,{"PageTaggedEntitiesExp::DocumentId": docid})
#                 for key in formatted.keys():
#                     if key in formatted_doc:
#                         formatted_doc[key] = formatted_doc[key] + formatted[key]
#                     else:
#                         formatted_doc[key] = formatted[key]
            except:
                print(traceback.format_exc())
                pass
        return formatted_doc
    except:
        print(traceback.format_exc())
        pass
def process_tagged_with_text(page):
    # extract all named entities
    tagged_entities = []
    index_count = 0
    entity_id = ''
    
    for term, tag in sentence:
        if tag != 'O':
            word = term
            word_tag = tag
            entity_id = uuid.uuid4()
            make_entity = {'entity_id': entity_id.hex, 'text': word, 'entity_score': 0.9902280569076538 , 'entity_type': word_tag,'string_index': index_count }                    
            index_count = len(term)+index_count+1
            tagged_entities.append(make_entity)
        else:
            index_count = len(term)+index_count+1
        
    return tagged_entities

def find_untagged_words(untagged,tagged):
    temp_tagged.append(tagged)
    temp_untagged.append(untagged)

def get_bucket_key(path):
    bucket = path.split('/')[2]
    key = path.replace(f'S3://{bucket}/', '')
    return bucket, key


def get_object(path, s3):
    bucket, key = get_bucket_key(path)
    res = s3.get_object(
        Bucket=bucket,
        Key=key
    )['Body'].read().decode('utf-8')
    return res


def put_object(path, s3, data):
    bucket, key = get_bucket_key(path)
    s3.put_object(
        Bucket=bucket,
        Body=json.dumps(data),
        Key=key
    )
def get_tagged_words(tagged):
    list_of_tagged_word_ids = []
    for page in tagged:
        rip_a_page = json.loads(page["body"])
        for entity in rip_a_page["entities"]:
            list_of_tagged_word_ids.append(entity["word_id"])
    return list_of_tagged_word_ids    
    
def get_untagged_words(untagged,list_of_tagged_word_ids):
    list_of_untagged_word_ids = []
    list_of_untagged_entities = []
    for page in untagged:
        for entity in page:
            list_of_untagged_word_ids.append(entity["word_id"])
    l3 = [x for x in list_of_untagged_word_ids if x not in list_of_tagged_word_ids]
    for word in l3:
        for page in untagged:
            for entity in page:
                if entity["word_id"] == word:
                    list_of_untagged_entities.append(entity)
    return list_of_untagged_entities,l3
    
def memorize_results_update_inplace(docid):
    formatted_doc = aggregate_formatted_entities(docid)
    current_path = remember_recall(docid, '_aggregated_formatted_entities_path')
    new_path = current_path.replace("FormattedEntities", f"FormattedEntities{exp_id}")
    put_object(new_path, s3, formatted_doc)
    return new_path
        
    

In [16]:
def call_make_df(parsed_words,blob,page_num,docid):
    for i in parsed_words:
        i["text_blob"] = blob
        i["page_ind"] = f'page_{page_num+1}'
        i["rid"] = docid
        i["page"] = page_num+1
        full_lits.append(i)
    

In [17]:
full_lits = []

In [18]:
df = pd.read_csv("1003_rid_new.csv",names=["rid"])

In [19]:
df.shape

(1601, 1)

In [20]:
temp = df.head(10)

In [21]:

#np.savetxt(r'testing_rids.txt', temp.rid.values, fmt='%s')


In [22]:
#temp[f"{exp_id}_path"] = temp.apply(lambda row: memorize_results_update_inplace(row["rid"]), axis = 1)

In [23]:

aws_json = aggregate_formatted_entities("18297279-7cb8-4a7f-bc85-d133659f0816")

pages--> [0]
[{'word_id': 'word_1_1', 'text': 'Uniform', 'string_index': 0, 'bounding_box': [520, 164, 779, 218], 'confidence': 0.9}, {'word_id': 'word_1_2', 'text': 'Underwriting', 'string_index': 8, 'bounding_box': [803, 164, 1208, 233], 'confidence': 0.92}, {'word_id': 'word_1_3', 'text': 'and', 'string_index': 21, 'bounding_box': [1235, 166, 1339, 219], 'confidence': 0.93}, {'word_id': 'word_1_4', 'text': 'Transmittal', 'string_index': 25, 'bounding_box': [1364, 164, 1709, 218], 'confidence': 0.91}, {'word_id': 'word_1_5', 'text': 'Summary', 'string_index': 37, 'bounding_box': [1736, 166, 2023, 233], 'confidence': 0.9}, {'word_id': 'word_1_6', 'text': 'I.', 'string_index': 45, 'bounding_box': [125, 264, 144, 293], 'confidence': 0.34}, {'word_id': 'word_1_7', 'text': 'Borrower', 'string_index': 48, 'bounding_box': [165, 264, 338, 294], 'confidence': 0.91}, {'word_id': 'word_1_8', 'text': 'and', 'string_index': 57, 'bounding_box': [355, 264, 421, 294], 'confidence': 0.91}, {'word_id'

In [13]:
for i,j in temp.iterrows():
    aws_json = aggregate_formatted_entities(j.rid)

pages--> [0, 1, 2, 3, 4]
pages--> [0, 1, 2, 3, 4]
pages--> [0, 1, 2, 3]
pages--> [0, 1, 2]
pages--> [0, 1, 2, 3]
pages--> [0, 1, 2]
pages--> [0, 1, 2, 3, 4, 5]
pages--> [0, 1, 2, 3, 4]
pages--> [0, 1, 2, 3, 4]
pages--> [0, 1, 2, 3]


In [ ]:
aws_json = aggregate_formatted_entities("bd5ff6ce-f0cf-4b16-8e75-5a25b09b6ad6")

In [ ]:
pd.DataFrame(full_lits)

In [22]:
training_frame = pd.DataFrame(full_lits)

In [23]:
training_frame
#training_frame = training_frame[training_frame["page"]<8]

,word_id,text,string_index,bounding_box,confidence,text_blob,page_ind,rid,page
0,word_1_1,Yellowstone,0,"[1139, 84, 1329, 109]",0.91,Yellowstone Bank Uniform Residential Loan Appl...,page_1,bd5ff6ce-f0cf-4b16-8e75-5a25b09b6ad6,1
1,word_1_2,Bank,12,"[1342, 84, 1420, 109]",0.92,Yellowstone Bank Uniform Residential Loan Appl...,page_1,bd5ff6ce-f0cf-4b16-8e75-5a25b09b6ad6,1
2,word_1_3,Uniform,17,"[691, 150, 936, 201]",0.92,Yellowstone Bank Uniform Residential Loan Appl...,page_1,bd5ff6ce-f0cf-4b16-8e75-5a25b09b6ad6,1
3,word_1_4,Residential,25,"[962, 150, 1308, 201]",0.91,Yellowstone Bank Uniform Residential Loan Appl...,page_1,bd5ff6ce-f0cf-4b16-8e75-5a25b09b6ad6,1
4,word_1_5,Loan,37,"[1336, 151, 1486, 201]",0.91,Yellowstone Bank Uniform Residential Loan Appl...,page_1,bd5ff6ce-f0cf-4b16-8e75-5a25b09b6ad6,1
...,...,...,...,...,...,...,...,...,...
1512285,word_8_140,07/05,826,"[857, 3094, 918, 3112]",0.88,Continuation Sheet - Schedule of Liabilities B...,page_8,71e84b42-1b13-4dd1-bcd6-cdc8a798c31f,8
1512286,word_8_141,rev.,832,"[927, 3094, 969, 3117]",0.79,Continuation Sheet - Schedule of Liabilities B...,page_8,71e84b42-1b13-4dd1-bcd6-cdc8a798c31f,8
1512287,word_8_142,6/09,837,"[984, 3094, 1038, 3118]",0.88,Continuation Sheet - Schedule of Liabilities B...,page_8,71e84b42-1b13-4dd1-bcd6-cdc8a798c31f,8
1512288,word_8_143,www_bytesoftware.com,842,"[1959, 3097, 2210, 3121]",0.47,Continuation Sheet - Schedule of Liabilities B...,page_8,71e84b42-1b13-4dd1-bcd6-cdc8a798c31f,8


In [25]:
training_frame.to_csv("300_final_dataset.csv",index=False)

In [ ]:
list_of_tagged_word_ids = get_tagged_words(temp_tagged)
list_of_untagged_entities, list_of_untagged_word_ids = get_untagged_words(temp_untagged,list_of_tagged_word_ids)
list_of_untagged_entities

In [ ]:
untagged_word_ids = [ent["word_id"] for ent in list_of_untagged_entities]


# Now Tagging experimentation with other taggers

In [ ]:
tagged_snre_pages = []
for blob_by_page in page_blobs:
    temp_tag = [sner_tagger.tag(word_tokenize(blob_by_page))]
    temp_result_array = process_tagged_with_text(temp_tag)
    tagged_snre_pages.append(temp_result_array)

In [ ]:
zipped_sner_tagged_entities = tagger.zip_words_entities(temp_untagged[0],tagged_snre_pages[0])

In [ ]:
comparelist = [word["word_id"] for word in zipped_sner_tagged_entities["entities"]]

In [ ]:
comparelist

In [ ]:
page_blobs[0]

In [ ]:
from spacy.lang.en import English
parser = English()

In [ ]:
def entities(example, show=False):
    if show: print(example)
    parsedEx = parser(example)
 
    print("-------------- entities only ---------------")
    # if you just want the entities and nothing else, you can do access the parsed examples "ents" property like this:
    ents = list(parsedEx.ents)
    tags={}
    for entity in ents:
        #print(entity.label, entity.label_, ' '.join(t.orth_ for t in entity))
        term=' '.join(t.orth_ for t in entity)
        if ' '.join(term) not in tags:
            tags[term]=[(entity.label, entity.label_)]
        else:
            tags[term].append((entity.label, entity.label_))
    print(tags)

In [ ]:
just_check.street_addresses

In [ ]:
import datefinder

In [ ]:
matches = datefinder.find_dates("If this is an application for joint credit, Borrower and Co-Borrower each agree that we intend to apg_ly for joint credit sign below: DocuSigned by: DocuSigned by —thu'tl M Jolunson. $020312  ")
for match in matches:
    print(match)